In [1]:
import math
from sortedcontainers import SortedDict
import random
import numpy 
import matplotlib.pyplot as plt

In [2]:
# Global Parameters:
t_max = 12*30 #*30 #352 months in hrs (assume no nights so 12 hr = 1 day)
timeline = SortedDict() #New and improved timeline (using a sorted dictionary)!
FV_param = [0, t_max] #5days, tmax/3 #0.2 #males/0.19 #totally random
FG_param = 3 #larger values lead to foraging ending earlier, because taus are smaller
SB_param = 1.5 #larger values lead to bowerstay ending earlier, because taus are smaller
RB_param = 6 #made up -- takes 30mins (every time -- no distribution) to repair bower
MA_param = 0.1 #made up -- takes 6mins (every time -- no distribution) to maraud bower
males = 2
next_forage = numpy.random.normal(loc=40000000, scale=.167, size=males)
bower_states = [0] * males
male_states = [0] * males 
strategy_states=[0]*males
mating_states = [0] * males
fitness_states = numpy.array([0] * males)
success_rate = 1
success_times = [] #going to change this to be a vector of times when the female leaves (at this point, sucessful mating has occured)
recents_list = []
max_visits = 6 #STEEEEVEEEE
bird_speed = 12 * 3600 #m/hr (12m/s)
x_dim, y_dim = 400, 500 #dimensions of environment
FperM = 3 #number of sexually mature females per sexually mature males in a bowerbird population
num_sims = 1
not_maraud_time = 0.15

new_tic_glob={}

d={} 

for i in range(males):
    d["next_forages{0}".format(i)]=[]
    d["forage_times{0}".format(i)]=[]

In [3]:
def ticketgenerator(t, tau, ow, ac, targ):
    ticket = {
        'time': t, #time at which the activity starts
        'tau': tau, #how long the activity lasts
        'owner': ow, #who is doing the activity (could be a female if action=-3)
        'action': ac, #which activity (see key)
        'target': targ, #target of the activity (owner=target unless action=-3,-4,-5, or -6)
    }
    return ticket

In [4]:
#Adds tickets to timeline- sorts by time
def addtotimeline(ticket):
    global timeline
    timeline[(ticket["time"], ticket["owner"])] = ticket

In [5]:
# creation of the environment

def envgenerator(males, x_dim, y_dim):
    Xs = numpy.random.rand(males) * x_dim
    Ys = numpy.random.rand(males) * y_dim
    return [Xs,Ys]

def dists(xy, bird_speed):
    male_dist = numpy.zeros(males * males).reshape((males, males))
    travel_times = numpy.zeros(males * males).reshape((males, males))
    for i in range(males):
        for j in range(i + 1, males):
            dist = math.sqrt((xy[0][j] - xy[0][i]) ** 2 + (xy[1][j] - xy[1][i]) ** 2)
            travel = dist / bird_speed
            male_dist[j][i] = dist
            male_dist[i][j] = dist
            travel_times[j][i] = travel
            travel_times[i][j] = travel
    return [male_dist,travel_times]

# the probability of choosing a neighbor at distance x is proportional to exp(-\lambda x)
# choose lambda such that 99% of the mass is before 800 meters
improb = 0.99
improb_distance = 800
lamb = -math.log(1 - improb) / improb_distance

#will write female preference based on cumulative exponential decay (lambda=.00576)
def preferences(male_dist, males, lamb):
    # compute exponential of each coefficient
    visit_preferences = numpy.exp(-lamb * male_dist)
    # remove the identity matrix (exp(0) = 1)
    visit_preferences = visit_preferences - numpy.eye(males)
    # make rows sum to one
    visit_preferences = (visit_preferences.transpose() / numpy.sum(visit_preferences, 1)).transpose()
    return visit_preferences

In [6]:
def nextFGtaugenerator(): #This generates the time until next_forage
    interval=-1
    while interval<=0: #ensures we pick positive values until next_forage
        interval=numpy.random.normal(loc=.4, scale=.167, size=1)[0]
    return interval
    

# function for determining the next time based on our rate parameters
def nexttau(action, ow, targ): 
    new_tau=-1
    while new_tau<=0: #ensures we pick positive values for new_tau
        switcher = { 
            -1: numpy.random.normal(loc=.1583, scale=.09755, size=1)[0], #choose when to leave bower (generate a tau for bower stay)
            -2: numpy.random.gamma(shape=1.5, scale=5, size=1)[0]/60, #choose when to stop foraging (generate a tau for foraging)
            -3: numpy.random.uniform(FV_param[0], FV_param[1]), #FV_param... totally arbitrary so we should think about it
            -4: travel_times[ow][targ],
            -5: MA_param, #in the future we'll do something with it
            -6: travel_times[targ][ow],
            -7: numpy.random.normal(loc=.1583, scale=.09755, size=1)[0],
        }
        new_tau=switcher.get(action)
    return new_tau

def futurebuilder(old_tic, new_ac, new_targ): #returns new ticket after ensuring it doesn't exceed t_max
    old_t=old_tic['time']
    new_tic={}
    if math.isclose(old_t, t_max, rel_tol=0, abs_tol=.0000001)==0: #if old_t is not essentially t_max
        ow=old_tic['owner']
        new_tau=nexttau(new_ac, ow, new_targ)
        end_t=old_t+new_tau #this time is the time when the action has ended
        if end_t>t_max: 
            new_tau=new_tau-(end_t-t_max) #modify the new_tau so end_t will be t_max
            end_t=old_t+new_tau #recalculate end_t
        new_tic=ticketgenerator(end_t, new_tau, ow, new_ac, new_targ) #now the time represents the end time for the action
    return new_tic

In [7]:
def statechanger(new_tic): #calls functions to change states of males i.e. for bower destruction, absence from bower
    options = {-1: SBstatechanger,
               -2: FGstatechanger,
               -3: FVstatechanger,
               -4: MTstatechanger,
               -5: MAstatechanger, 
               -6: MRstatenotchanger,
               -7: RBstatechanger
              }
    options[new_tic['action']](new_tic)
    
def SBstatechanger(new_tic): 
    ow=new_tic['owner']
    t=new_tic['time']
    male_states[ow]=t #male is at his bower / store time so that we FV can access and modify SB_tic in sorted dict

def FGstatechanger(new_tic):
    ow=new_tic['owner']
    male_states[ow]=0 #Male is now absent from bower

def FVstatechanger(new_tic): #here the t is the start time... idea for future: female start and end tics?
    targ=new_tic['target']
    t=new_tic['time']
    if bower_states[targ]==0 and male_states[targ]!=0 and mating_states[targ]<t:
        fitness_states[targ]+=1
        mating_end=.25+t #can use dist instead of .25
        if t<mating_end:
            SB_tic=tl_dict[t] #PSEUDO
            SB_tic['tau']=SB_tic['tau']+(mating_end-SB_tic['time'])
            SB_tic['time']=mating_end
            #re-add tic to sorted dict (if not done automatically)
        mating_states[targ]=mating_end      
    
def MTstatechanger(new_tic):
    ow=new_tic['owner']
    male_states[ow]=0  #Male is now absent from bower

def MAstatechanger(new_tic): #store number of successful marauds of each male --> MAKE A NEW STATE
    targ=new_tic['target']
    if bower_states[targ]==0 and male_states[targ]==0:
        bower_states[targ]=-1*RB_param
    #MAKE IT EITHER TAKE maraud_time or 0 (in the else)
    
def MRstatenotchanger(new_tic):
    pass

def RBstatechanger(new_tic):
    tau=new_tic['tau']
    if bower_states[ow]+tau>0:
        new_tic['time']-=bower_states[ow]+tau #think about it more in the AM :)
        tau=-1*bower_states[ow]
        new_tic['tau']=tau
        #add changes to timeline dict if necessary
    bower_states[ow]+=tau
    male_states[ow]=new_tic['time'] #although it doesn't matter from an FV POV
    

#choose what action / sequence of actions to do next
def decider(tic):
    ow = tic['owner']
    ac = tic['action'] 
    t = tic['time']
    targ=ow #will be reset in the case of -4
    if t>next_forage[ow]:
        d["forage_times{0}".format(ow)].append(t)
        #print("t is greater than next_FG[{:f}]: t={:f}, next_FG={:f}".format(ow, t, next_forage[ow]))
        new_ac=-2 #forage
        d["next_forages{0}".format(ow)].append(next_forage[ow])
        next_forage[ow] = t+nextFGtaugenerator() #forage 30 times / day on average (but underestimates)
    elif bower_states[ow]<0:
        new_ac=-7 #repair bower
    else:
        if random.random()<strategy_states[ow]: #if male chooses to maraud 
            targ=numpy.random.choice(list(range(males)), p=visit_preferences[ow])
            new_ac=-4 #maurad travel
        else: #if male chooses to stay at bower
            new_ac=-1 #stay at bower
    new_tic=futurebuilder(tic, new_ac, targ) #ONE IDEA -- separate ticket generation and adding to timeline
    return new_tic

In [8]:
xy = envgenerator(males, x_dim, y_dim)

travel_mats=dists(xy, bird_speed)
male_dist=travel_mats[0]
travel_times=travel_mats[1]

visit_preferences=preferences(male_dist, males, lamb)

In [9]:
for i in range(males):
    addtotimeline(ticketgenerator(0.0, 0.0, i, -2, i))

In [10]:
acc=len(timeline)
new_tic=ticketgenerator(0.0, 0.0, 0, -2, 0) #for testing
for key in timeline:
    acc+=1
    tic=timeline[key]
    #if tic!=new_tic:
        #print("error^")
        #print(tic)
        #print(new_tic)
    #print("this")
    #print(tic)
    ac=tic['action']
    ow=tic['owner']
    targ=tic['target']
    if ac==-4 or ac==-5:
        #generate a -5 or -6 tic as new_tic and pass it in
        new_tic=futurebuilder(tic, ac-1, targ)
    #elif ac==-3:
    #    #d["rl{0}".format(ow)]=FVtickethandler(next_tic, d["rl{0}".format(ow)])
    #    new_tic=FVtickethandler()
    else:
        new_tic=decider(tic)
    if new_tic=={}:
        print("done (for some owner)!")
    else:
        statechanger(new_tic)
        #print("produces this:")
        #print(new_tic)
        addtotimeline(new_tic)

KeyboardInterrupt: 

In [ ]:
print(len(timeline)/(males*t_max))

In [11]:
len(timeline)

693937

In [ ]:
timeline

In [ ]:
timeline.popitem()

In [ ]:
?timeline.popitem()